In [70]:
from datetime import datetime

In [2]:

from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType

feats = []
f = open('features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

print(schema)

StructType(List(StructField(Timestamp,LongType,true),StructField(Geohash,StringType,true),StructField(geopotential_height_lltw,FloatType,true),StructField(water_equiv_of_accum_snow_depth_surface,FloatType,true),StructField(drag_coefficient_surface,FloatType,true),StructField(sensible_heat_net_flux_surface,FloatType,true),StructField(categorical_ice_pellets_yes1_no0_surface,FloatType,true),StructField(visibility_surface,FloatType,true),StructField(number_of_soil_layers_in_root_zone_surface,FloatType,true),StructField(categorical_freezing_rain_yes1_no0_surface,FloatType,true),StructField(pressure_reduced_to_msl_msl,FloatType,true),StructField(upward_short_wave_rad_flux_surface,FloatType,true),StructField(relative_humidity_zerodegc_isotherm,FloatType,true),StructField(categorical_snow_yes1_no0_surface,FloatType,true),StructField(u-component_of_wind_tropopause,FloatType,true),StructField(surface_wind_gust_surface,FloatType,true),StructField(total_cloud_cover_entire_atmosphere,FloatType,tru

In [3]:

#df = spark.read.format('csv').option('sep', '\t').schema(schema).load('/Volumes/evo/Datasets/NAM_2015_S/*')
df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/nam_tiny.tdv')
# df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201501.tdv.gz')


In [4]:
df_large = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201509.tdv.gz')


In [32]:
df_entire = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/*')


In [5]:
really_hot = df.filter(df.snow_depth_surface != 0).count()
print(really_hot)

hot_and_humid = df.filter(df.temperature_surface > 313).filter(df.relative_humidity_zerodegc_isotherm > .8).count()
print(hot_and_humid)

25
0


In [8]:
# df.filter(df.snow_depth_surface != 0).take(5)

[Row(Timestamp=1426377600000, Geohash='cf7ecr4h2ps0', geopotential_height_lltw=136.53125, water_equiv_of_accum_snow_depth_surface=77.0, drag_coefficient_surface=0.0, sensible_heat_net_flux_surface=-39.57763671875, categorical_ice_pellets_yes1_no0_surface=0.0, visibility_surface=24221.587890625, number_of_soil_layers_in_root_zone_surface=3.0, categorical_freezing_rain_yes1_no0_surface=0.0, pressure_reduced_to_msl_msl=99602.0, upward_short_wave_rad_flux_surface=6.625, relative_humidity_zerodegc_isotherm=34.0, categorical_snow_yes1_no0_surface=0.0, u-component_of_wind_tropopause=27.527877807617188, surface_wind_gust_surface=16.158788681030273, total_cloud_cover_entire_atmosphere=100.0, upward_long_wave_rad_flux_surface=314.0560302734375, land_cover_land1_sea0_surface=1.0, vegitation_type_as_in_sib_surface=18.0, v-component_of_wind_pblri=12.31915283203125, albedo_surface=38.75, lightning_surface=0.0, ice_cover_ice1_no_ice0_surface=0.0, convective_inhibition_surface=-0.65234375, pressure_su

# Yifan Zhou's Warn_up

## Q1. Unknown Feature: Choose a feature from the data dictionary above that you have never heard of before. Inspect some of the values for the feature (such as its average, min, max, etc.) and try to guess what it measures. Was your hypothesis correct?

In [9]:
from pyspark.sql.functions import avg

df.select(avg(df.geopotential_height_pblri)).show()

+------------------------------+
|avg(geopotential_height_pblri)|
+------------------------------+
|              406.451460521314|
+------------------------------+



This attribute may records geopotential height for each loaction.

## Q2. Hot hot hot: When and where was the hottest temperature observed in the dataset? Is it an anomaly?

In [64]:
# Creating an SQL 'table'
df_entire.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
hot = spark.sql("SELECT Geohash,temperature_surface FROM TEMP_DF WHERE temperature_surface IN (SELECT MAX(temperature_surface) FROM TEMP_DF) ").collect()

for x in hot:
    print(x)


Row(Geohash='d5dpds10m55b', temperature_surface=331.390625)


The hottest temperature in this dataset is 331.39F (166.3C). Obviously, Human cannot live in such hot place.

## Q3. So Snowy: Find a location that is snowy all year (there are several). Locate a nearby town/city and provide a small writeup about it. Include pictures if you’d like.

In [ ]:
# Creating an SQL 'table'
df_large.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
snow = spark.sql("SELECT Geohash FROM TEMP_DF WHERE Geohash NOT IN (SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0) ").collect()
for x in snow:
    print(x)

In [123]:
# Creating an SQL 'table'
df_entire.createOrReplaceTempView("TEMP_DF")
started_at = datetime.now()
# Let's get all the snow cover values:
# not_snow_all_year = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0 ").collect()
# all_Geohash = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF").collect()
# snow = spark.sql("SELECT DISTINCT Geohash from TEMP_DF group by Geohash having avg(snow_cover_surface) >= 100").collect()
snow = spark.sql("SELECT DISTINCT Geohash, avg(categorical_snow_yes1_no0_surface) as avgSnow from TEMP_DF group by Geohash order by avgSnow DESC limit 5").collect()
print('Total snowy location count: %d' %len(snow))
for x in snow:
    print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Total snowy location count: 5
Row(Geohash='c43k6uu1egxb', avgSnow=0.37908011869436203)
Row(Geohash='c43kcu3t702p', avgSnow=0.37908011869436203)
Row(Geohash='c41ueb1jyypb', avgSnow=0.3775964391691395)
Row(Geohash='c41uhb4r5n00', avgSnow=0.3775964391691395)
Row(Geohash='c41v48pupf00', avgSnow=0.37537091988130566)
Finished. it's been 520 seconds


# Analysis

## Q1. Strangely Snowy: Find a location that contains snow while its surroundings do not. Why does this occur? Is it a high mountain peak in a desert?

In [7]:
df_large.createOrReplaceTempView("TEMP_DF")
not_snow_all_year = spark.sql("SELECT  Geohash,avg(snow_cover_surface) FROM TEMP_DF group by Geohash having avg(snow_cover_surface) >= 99").collect()

for x in not_snow_all_year:
    print(x)
print(len(not_snow_all_year))

Row(Geohash='c1gxsefbz52p', avg(snow_cover_surface)=100.0)
Row(Geohash='c43hd76g0npb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43g3722yrzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c1nt970j5b5b', avg(snow_cover_surface)=100.0)
Row(Geohash='c1grzzxwt1bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c436f3k4p8xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43jhzrrsbrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c44hk1hy9u2p', avg(snow_cover_surface)=100.0)
Row(Geohash='c46sf3s53p80', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gkzxxn6bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c41tugrtwhzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c458r20zx080', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gb4ywdgrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c438x5esgf00', avg(snow_cover_surface)=100.0)
Row(Geohash='c4459s8rz1xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43c1zbmntbp', avg(snow_cover_surface)=100.0)
Row(Geohash='c43e1f64v9bp', avg(snow_cover_surface)=100.

In [8]:
four_set = set()
for x in not_snow_all_year:
    print(x.Geohash[0:4])
    four_set.add(x.Geohash[0:4])
print(len(not_snow_all_year))
print(len(four_set))

c1gx
c43h
c43g
c1nt
c1gr
c436
c43j
c44h
c46s
c43g
c41t
c458
c43g
c438
c445
c43c
c43e
c41z
c43u
c44n
c41u
c439
c1q2
c1nt
c432
c43e
c1gr
c44n
c1gn
c43e
c41z
c452
c1nm
c1nr
c44p
c43c
c304
c43b
c460
c43m
c43h
c1gy
c43k
c1gr
c43d
c1gz
c1gx
c43u
c44p
c43d
c41t
c433
c44j
c1q0
c41x
c436
c438
c1pc
c1gx
c43f
c437
c439
c43d
c459
c1nq
c43c
c1nk
c1p5
c460
c436
c43s
c1gz
c1p8
c464
c41v
c3k8
c43k
c45b
c41u
c3hz
c41u
c304
c1pe
c41w
c445
c438
c3k0
c3k3
c43b
c41w
c436
c43h
c460
c43u
c458
c307
c458
c1pf
c437
c464
c44h
c36u
c41g
c44p
c1uh
c44h
c1gy
c43g
c3k2
c461
c41x
c41z
c304
c1nu
c41y
c1gy
c41z
c41w
c301
c36u
c1sn
c43j
c44h
c44w
c43k
c464
c1gq
c1gw
c301
c465
c44n
c43e
c43f
c439
c1nk
c1gz
c439
c1gr
c41v
c43s
c43h
c1sn
c41y
c1gq
c41v
c43f
c4r5
c1p4
c1gz
149
74


In [24]:
# df.createOrReplaceTempView("TEMP_DF")
groupList = list()
for y in four_set:
        param = y + '%'
        group = spark.sql("SELECT distinct Geohash,avg(snow_cover_surface) as avgSnow FROM TEMP_DF WHERE Geohash like '%s' group by Geohash" %param ).collect()
        print(param ) 
        print(len(group))
        groupList.append(group)
        
print(len(groupList))
print('Complete')


c3hz%
4
c43g%
5
c43m%
3
c1p4%
5
c41z%
4
c36u%
4
c1nr%
4
c460%
3
c1p8%
3
c41g%
4
c436%
4
c1nt%
4
c433%
4
c41x%
4
c452%
3
c1q0%
4
c3k8%
4
c43e%
4
c43f%
3
c1uh%
4
c458%
4
c1nm%
3
c41y%
3
c43b%
5
c459%
4
c43u%
4
c1pf%
3
c3k3%
4
c44n%
4
c1gz%
4
c1p5%
3
c44h%
5
c1pc%
4
c1nu%
4
c432%
5
c44j%
4
c1nk%
5
c439%
4
c304%
5
c1gn%
4
c43s%
4
c3k0%
5
c43h%
4
c3k2%
3
c44w%
3
c43k%
4
c41w%
5
c461%
4
c43c%
4
c1gx%
3
c43d%
4
c1gr%
5
c44p%
4
c1nq%
4
c41v%
5
c1gq%
4
c1gw%
4
c41u%
4
c438%
3
c1gy%
4
c1sn%
4
c43j%
2
c41t%
4
c445%
4
c1q2%
5
c4r5%
4
c46s%
4
c45b%
4
c301%
4
c464%
5
c465%
4
c1pe%
4
c307%
4
c437%
4
74
Complete


In [26]:
for x in groupList:
    avg=0
    for y in x:
        avg= avg + y.avgSnow
    avg= avg/len(x)
    if avg<30:
        print(x)
#     print(avg)
        

[Row(Geohash='c1uhvnv5p8pb', avgSnow=7.283950617283951), Row(Geohash='c1uh66r1r02p', avgSnow=9.74074074074074), Row(Geohash='c1uhcd9k1n2p', avgSnow=100.0), Row(Geohash='c1uhqne01400', avgSnow=2.9135802469135803)]
[Row(Geohash='c3k0fgxqugeb', avgSnow=10.790123456790123), Row(Geohash='c3k0yw8m6us0', avgSnow=100.0), Row(Geohash='c3k0rj7e0zs0', avgSnow=21.506172839506174), Row(Geohash='c3k07d6swzkp', avgSnow=4.08641975308642), Row(Geohash='c3k00p3vw3gz', avgSnow=4.08641975308642)]


c1uhcd9k1n2p is a mountain peak(lavender peak)

## Q2. Lightning rod: Where are you most likely to be struck by lightning? Use a precision of at least 4 Geohash characters and provide the top 3 locations.

In [36]:
df_entire.createOrReplaceTempView("TEMP_DF")
started_at = datetime.now()
lightning = spark.sql("SELECT  Geohash,avg(lightning_surface) as avgLighting FROM TEMP_DF group by Geohash order by avgLighting DESC Limit 10 ").collect()

for x in lightning:
    print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Geohash='9eqepuxk7x20', avgLighting=0.3724035608308605)
Row(Geohash='9g3h968ygj7z', avgLighting=0.36053412462908013)
Row(Geohash='9g3y52sgeceb', avgLighting=0.3508902077151335)
Row(Geohash='9g3ug8ckk4hp', avgLighting=0.3486646884272997)
Row(Geohash='9g3m79nrf3zb', avgLighting=0.3479228486646884)
Row(Geohash='9g3mq3f7y6eb', avgLighting=0.3479228486646884)
Row(Geohash='9g3v7kxpuhh0', avgLighting=0.3464391691394659)
Row(Geohash='d7mkkfvu34up', avgLighting=0.336053412462908)
Row(Geohash='9g2vntcg8c2p', avgLighting=0.3353115727002967)
Row(Geohash='9g35cqggdn5z', avgLighting=0.33456973293768544)
10


I calculate the average lightning chances. The top 3 location is 9eqe, 9g3h, 9g3y

## Q3. Drying out: Choose a region in North America (defined by one or more Geohashes) and determine when its driest month is. This should include a histogram with data from each month.

In [57]:
dfList = list()
for x in range(1,13):
    if x<10:
        x = '0'+str(x)
    path = "hdfs://orion11:40910/datasets/nam_2015%s.tdv.gz" %x
#     print(path)
    df_1 = spark.read.format('csv').option('sep', '\t').schema(schema).load(path)
    dfList.append(df_1)
print(len(dfList))

12


In [58]:
#9q9n categorical_rain_yes1_no0_surface
from pyspark.sql.functions import substring
started_at = datetime.now()
for df_1 in dfList:  
    df_1.createOrReplaceTempView("TEMP_DF")
    dry_place = spark.sql("SELECT  substring(Geohash, 1, 4) as subGeo, AVG(categorical_rain_yes1_no0_surface) as avgRain FROM TEMP_DF group by subGeo having subGeo = '9q9n'").collect()
    for x in dry_place:
        print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(subGeo='9q9n', avgRain=0.0)
Row(subGeo='9q9n', avgRain=0.0603448275862069)
Row(subGeo='9q9n', avgRain=0.0)
Row(subGeo='9q9n', avgRain=0.0777027027027027)
Row(subGeo='9q9n', avgRain=0.020161290322580645)
Row(subGeo='9q9n', avgRain=0.0022935779816513763)
Row(subGeo='9q9n', avgRain=0.01639344262295082)
Row(subGeo='9q9n', avgRain=0.008333333333333333)
Row(subGeo='9q9n', avgRain=0.009259259259259259)
Row(subGeo='9q9n', avgRain=0.0021551724137931034)
Row(subGeo='9q9n', avgRain=0.0546218487394958)
Row(subGeo='9q9n', avgRain=0.0872093023255814)


This is the avgerage rain chances for Hayward. January, March do not rain at all. [Bay Area](bayArea.jpg)

## Q5. Escaping the fog: After becoming rich from your startup, you are looking for the perfect location to build your Bay Area mansion with unobstructed views. Find the locations that are the least foggy and show them on a map.

In [71]:
started_at = datetime.now()
df_entire.createOrReplaceTempView("TEMP_DF")
visibility = spark.sql("SELECT  Geohash,avg(visibility_surface) as avgVisibility FROM TEMP_DF WHERE Geohash like '9q9p%' or Geohash like '9q9n%' or Geohash like '9q9j%' or Geohash like '9q9m%' or Geohash like '9q9k%' or Geohash like '9q9h%' or Geohash like '9q8v%' or Geohash like '9q8y%' or Geohash like '9q8z%' group by Geohash order by avgVisibility DESC ").collect()
for x in visibility:
    print(x)
print(len(visibility))
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Geohash='9q9jnpfx5rxb', avgVisibility=23811.522961255705)
Row(Geohash='9q9jv7p453zz', avgVisibility=23703.065985376827)
Row(Geohash='9q9m35ffeg7z', avgVisibility=23698.021476396112)
Row(Geohash='9q9kc8wprss0', avgVisibility=23675.469547196735)
Row(Geohash='9q9jdrny12rz', avgVisibility=23616.419106960297)
Row(Geohash='9q8zuzx3hzeb', avgVisibility=23600.692098283627)
Row(Geohash='9q9k4m9efqh0', avgVisibility=23497.428004600173)
Row(Geohash='9q9hp3c2xy5b', avgVisibility=23449.653520560052)
Row(Geohash='9q9hwsyh477z', avgVisibility=23436.152038374712)
Row(Geohash='9q9mbqzmqxrz', avgVisibility=23435.8553033342)
Row(Geohash='9q9pesk2gbs0', avgVisibility=23400.098629552464)
Row(Geohash='9q9j5h47k07z', avgVisibility=23329.623851402575)
Row(Geohash='9q9py8hurvup', avgVisibility=23277.991815833026)
Row(Geohash='9q9nky5sd52p', avgVisibility=23164.93542607814)
Row(Geohash='9q9nxfh005kp', avgVisibility=23088.525926889935)
Row(Geohash='9q9p887u91up', avgVisibility=22978.882014243467)
Row(Geohash

The list above has the best visibility in the bay area. 
Such as 9q9m35 is a good place to build a mansion. The address is S Grimmer Blvd:Fremont Blvd, Fremont, CA
[Grimmer](Grimmer.jpg)

# Option 1

In [120]:
df_option = spark.read.option("header","true").option("inferSchema","true").csv('hdfs://orion11:40910/option_dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')


In [118]:
notComplete = df_option.filter(df_option.Category == "ASSAULT").count()
print(notComplete)

194694


In [124]:
df_option.createOrReplaceTempView("POLICE_DF")
spark.sql("select * from POLICE_DF limit 1").show()


+----------+--------+--------+---------+----------+-----+----------+--------------+--------------------+----------------+---------------+--------------------+--------------+
|IncidntNum|Category|Descript|DayOfWeek|      Date| Time|PdDistrict|    Resolution|             Address|               X|              Y|            Location|          PdId|
+----------+--------+--------+---------+----------+-----+----------+--------------+--------------------+----------------+---------------+--------------------+--------------+
| 140646669| ASSAULT| BATTERY|   Monday|08/04/2014|08:56|   MISSION|ARREST, BOOKED|500 Block of SOUT...|-122.41747701285|37.764357751686|(37.764357751686,...|14064666904134|
+----------+--------+--------+---------+----------+-----+----------+--------------+--------------------+----------------+---------------+--------------------+--------------+



## Q1. 2003-2017, 1- 24. batman

In [128]:
started_at = datetime.now()
df_option.createOrReplaceTempView("POLICE_DF")
batman = spark.sql("SELECT substring(time, 1, 2 ) hour, count(*) as timecount FROM POLICE_DF  group by hour order by hour").collect()
for x in batman:
    print(x)
print(len(batman))
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(hour='00', timecount=113096)
Row(hour='01', timecount=65182)
Row(hour='02', timecount=54550)
Row(hour='03', timecount=35596)
Row(hour='04', timecount=25285)
Row(hour='05', timecount=22413)
Row(hour='06', timecount=33494)
Row(hour='07', timecount=55551)
Row(hour='08', timecount=82459)
Row(hour='09', timecount=89303)
Row(hour='10', timecount=95469)
Row(hour='11', timecount=97620)
Row(hour='12', timecount=132631)
Row(hour='13', timecount=108540)
Row(hour='14', timecount=112078)
Row(hour='15', timecount=120190)
Row(hour='16', timecount=125548)
Row(hour='17', timecount=135481)
Row(hour='18', timecount=140918)
Row(hour='19', timecount=126404)
Row(hour='20', timecount=115010)
Row(hour='21', timecount=109559)
Row(hour='22', timecount=113915)
Row(hour='23', timecount=104732)
24
Finished. it's been 1 seconds


## Q2. Break in car (VANDALISM) when and where

In [134]:
started_at = datetime.now()
vandalism = spark.sql("SELECT PdDistrict, count(*) c FROM POLICE_DF WHERE Category ='VANDALISM' group by PdDistrict order by c").collect()
count = 0
for x in vandalism:
    count = count + x.c
    print(x)
print(len(vandalism))
print('There are totally ' + str(count) + ' vandalisms')
for y in vandalism:
    print(y.PdDistrict)
    print(y.c/count)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(PdDistrict='TENDERLOIN', c=4214)
Row(PdDistrict='PARK', c=6554)
Row(PdDistrict='RICHMOND', c=7837)
Row(PdDistrict='TARAVAL', c=11607)
Row(PdDistrict='CENTRAL', c=12586)
Row(PdDistrict='INGLESIDE', c=13126)
Row(PdDistrict='MISSION', c=14050)
Row(PdDistrict='BAYVIEW', c=14103)
Row(PdDistrict='NORTHERN', c=14533)
Row(PdDistrict='SOUTHERN', c=17449)
10
There are totally 116059vandalism
TENDERLOIN
0.03630911863793415
PARK
0.05647127753987196
RICHMOND
0.06752599970704555
TARAVAL
0.10000947793794536
CENTRAL
0.10844484270931164
INGLESIDE
0.11309764860975884
MISSION
0.12105911648385735
BAYVIEW
0.12151578076667902
NORTHERN
0.12522079287259066
SOUTHERN
0.15034594473500548
Finished. it's been 1 seconds


In [135]:
started_at = datetime.now()
vandalism = spark.sql("SELECT substring(time, 1, 2 ) hour, count(*) as timecount FROM POLICE_DF WHERE Category ='VANDALISM' group by hour order by hour").collect()
for x in vandalism:
    print(x)
print(len(vandalism))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(hour='00', timecount=6534)
Row(hour='01', timecount=4562)
Row(hour='02', timecount=4303)
Row(hour='03', timecount=2898)
Row(hour='04', timecount=1942)
Row(hour='05', timecount=1611)
Row(hour='06', timecount=1856)
Row(hour='07', timecount=2518)
Row(hour='08', timecount=3684)
Row(hour='09', timecount=3474)
Row(hour='10', timecount=3587)
Row(hour='11', timecount=3395)
Row(hour='12', timecount=4640)
Row(hour='13', timecount=3600)
Row(hour='14', timecount=4077)
Row(hour='15', timecount=4801)
Row(hour='16', timecount=5316)
Row(hour='17', timecount=7077)
Row(hour='18', timecount=8402)
Row(hour='19', timecount=7619)
Row(hour='20', timecount=7549)
Row(hour='21', timecount=7683)
Row(hour='22', timecount=7829)
Row(hour='23', timecount=7102)
24
Finished. it's been 1 seconds


看上去多发生在半夜

In [136]:
started_at = datetime.now()
vandalism = spark.sql("SELECT substring(date, 1, 2 ) dates, count(*) as timecount FROM POLICE_DF WHERE Category ='VANDALISM' group by dates order by dates").collect()
for x in vandalism:
    print(x)
print(len(vandalism))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(dates='01', timecount=9982)
Row(dates='02', timecount=8958)
Row(dates='03', timecount=10407)
Row(dates='04', timecount=10173)
Row(dates='05', timecount=9752)
Row(dates='06', timecount=9336)
Row(dates='07', timecount=10065)
Row(dates='08', timecount=9718)
Row(dates='09', timecount=9501)
Row(dates='10', timecount=10112)
Row(dates='11', timecount=9002)
Row(dates='12', timecount=9053)
12
Finished. it's been 1 seconds


月份上看起来挺平均的

In [137]:
started_at = datetime.now()
vandalism = spark.sql("SELECT substring(date, 7, 4 ) year, count(*) as timecount FROM POLICE_DF WHERE Category ='VANDALISM' group by year order by year").collect()
# vandalism = spark.sql("SELECT substring(time, 1, 2 ) hour, date, PdDistrict FROM POLICE_DF WHERE Category ='VANDALISM' order by hour").collect()
for x in vandalism:
    print(x)
print(len(vandalism))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(year='2003', timecount=6448)
Row(year='2004', timecount=6496)
Row(year='2005', timecount=7013)
Row(year='2006', timecount=7688)
Row(year='2007', timecount=7566)
Row(year='2008', timecount=7342)
Row(year='2009', timecount=7604)
Row(year='2010', timecount=7934)
Row(year='2011', timecount=7243)
Row(year='2012', timecount=7808)
Row(year='2013', timecount=6921)
Row(year='2014', timecount=7165)
Row(year='2015', timecount=7675)
Row(year='2016', timecount=8595)
Row(year='2017', timecount=9765)
Row(year='2018', timecount=2796)
16
Finished. it's been 1 seconds


年份上也很平均

## Q3. 
### a. Top 10 Category and resolution rate. 

In [147]:
started_at = datetime.now()
category = spark.sql("SELECT a.Category, b.totalCount, a.resolveCount from (SELECT distinct Category, count(*) as resolveCount FROM POLICE_DF WHERE not Resolution ='NONE' group by Category) a inner join (SELECT distinct Category, count(*) as totalCount FROM POLICE_DF group by Category order by totalCount DESC limit 10) b on a.Category = b.Category order by b.totalCount DESC").collect()
for x in category:
    print(x)
print(len(category))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Category='LARCENY/THEFT', totalCount=480448, resolveCount=42521)
Row(Category='OTHER OFFENSES', totalCount=309358, resolveCount=221514)
Row(Category='NON-CRIMINAL', totalCount=238323, resolveCount=53465)
Row(Category='ASSAULT', totalCount=194694, resolveCount=80947)
Row(Category='VEHICLE THEFT', totalCount=126602, resolveCount=10622)
Row(Category='DRUG/NARCOTIC', totalCount=119628, resolveCount=109357)
Row(Category='VANDALISM', totalCount=116059, resolveCount=14169)
Row(Category='WARRANTS', totalCount=101379, resolveCount=95897)
Row(Category='BURGLARY', totalCount=91543, resolveCount=14890)
Row(Category='SUSPICIOUS OCC', totalCount=80444, resolveCount=9458)
10
Finished. it's been 2 seconds


Top 10 category, and their resolution rates

In [142]:
started_at = datetime.now()
category = spark.sql("SELECT distinct Category, count(*) as count FROM POLICE_DF WHERE not Resolution ='NONE' group by Category order by count DESC limit 10").collect()
for x in category:
    print(x)
print(len(category))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Category='OTHER OFFENSES', count=221514)
Row(Category='DRUG/NARCOTIC', count=109357)
Row(Category='WARRANTS', count=95897)
Row(Category='ASSAULT', count=80947)
Row(Category='NON-CRIMINAL', count=53465)
Row(Category='LARCENY/THEFT', count=42521)
Row(Category='MISSING PERSON', count=34672)
Row(Category='WEAPON LAWS', count=16164)
Row(Category='PROSTITUTION', count=15851)
Row(Category='BURGLARY', count=14890)
10
Finished. it's been 1 seconds


### b. Relationship between criminal increase rate and solved criminal rate

In [146]:
started_at = datetime.now()
category = spark.sql("SELECT distinct Category, count(*) as totalCount FROM POLICE_DF group by Category order by totalCount DESC limit 10").collect()
for x in category:
    print(x)
print(len(category))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Category='LARCENY/THEFT', totalCount=480448)
Row(Category='OTHER OFFENSES', totalCount=309358)
Row(Category='NON-CRIMINAL', totalCount=238323)
Row(Category='ASSAULT', totalCount=194694)
Row(Category='VEHICLE THEFT', totalCount=126602)
Row(Category='DRUG/NARCOTIC', totalCount=119628)
Row(Category='VANDALISM', totalCount=116059)
Row(Category='WARRANTS', totalCount=101379)
Row(Category='BURGLARY', totalCount=91543)
Row(Category='SUSPICIOUS OCC', totalCount=80444)
10
Finished. it's been 1 seconds


In [157]:
started_at = datetime.now()
lists = list()
for x in category:
    types = x.Category
    yearResolution = spark.sql("SELECT a.year, a.totalCount, b.resolveCount from (SELECT substring(date, 7, 4 ) year, count(*) as totalCount FROM POLICE_DF WHERE Category = '%s' group by year) a inner join (SELECT substring(date, 7, 4 ) year, count(*) as resolveCount FROM POLICE_DF WHERE (not Resolution ='NONE') and Category = '%s' group by year )b on a.year = b.year order by a.year " %(types,types)).collect()
    print(types)
    lists.append(yearResolution)
    for x in yearResolution:
        print(str(x.year) +'  total = ' + str(x.totalCount) + '  resolution = '+str(x.resolveCount))
# for x in category:
#     print(x)
# print(len(category))

print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

LARCENY/THEFT
2003  total = 26393  resolution = 3844
2004  total = 24505  resolution = 3389
2005  total = 25319  resolution = 3346
2006  total = 27352  resolution = 3345
2007  total = 25770  resolution = 2879
2008  total = 25807  resolution = 3099
2009  total = 25585  resolution = 3014
2010  total = 24446  resolution = 2973
2011  total = 25905  resolution = 2729
2012  total = 30976  resolution = 2358
2013  total = 36412  resolution = 2777
2014  total = 38003  resolution = 2367
2015  total = 42068  resolution = 2135
2016  total = 40449  resolution = 1837
2017  total = 47826  resolution = 1882
2018  total = 13632  resolution = 547
OTHER OFFENSES
2003  total = 21232  resolution = 16295
2004  total = 20710  resolution = 15511
2005  total = 17834  resolution = 12901
2006  total = 18306  resolution = 12894
2007  total = 19763  resolution = 14910
2008  total = 23457  resolution = 18447
2009  total = 24693  resolution = 19452
2010  total = 20990  resolution = 15295
2011  total = 19552  resolut

In [162]:
for x in lists:
    for y in x:
        print(str(y.year) + '  ' + str(y.resolveCount/ y.totalCount))
    print('*****************')
    

2003  0.14564467851324214
2004  0.13829830646806773
2005  0.1321537185512856
2006  0.12229453056449255
2007  0.11171905316259216
2008  0.1200836982214128
2009  0.11780340042993942
2010  0.12161498813711855
2011  0.10534645821270025
2012  0.07612345041322315
2013  0.07626606613204438
2014  0.06228455648238297
2015  0.05075116478083103
2016  0.045415214220376275
2017  0.03935098063814661
2018  0.04012617370892019
*****************
2003  0.7674736247174077
2004  0.7489618541767262
2005  0.7233935179993272
2006  0.7043592264831203
2007  0.7544401153671001
2008  0.7864177004732062
2009  0.7877536143846434
2010  0.7286803239637922
2011  0.7245806055646481
2012  0.6669526976295184
2013  0.7144250513347022
2014  0.7014464802314369
2015  0.6578844078108135
2016  0.6416780943674133
2017  0.6178204848220136
2018  0.6073994252873564
*****************
2003  0.13263366035439958
2004  0.1505298301640296
2005  0.16549270722162931
2006  0.21289646918013166
2007  0.23309931371775658
2008  0.236771519141